In [ ]:
import requests
import pprint as pp
import pandas as pd
import sqlite3 as sql
import os

# API Call

In [ ]:
api_key = #add your own api key as a string
with open('origin_ids.txt') as file:
    origin = file.read()
with open('destination_ids.txt') as file:
    destination = file.read()
url = f"https://maps.googleapis.com/maps/api/distancematrix/json?origins={origin}&destinations={destination}&mode=car&key={api_key}"
output = requests.get(url).json()
pp.pprint(output)

# 'Add-location' function

In [ ]:
def add_location(document=str, location=str):
    location = location.replace("|", "")
    
    if not os.path.isfile(document):
        with open(document, 'a+') as file:
            file.write(location)
    else: 
        file = open(document)
        if file.read().count('|') >= 24:
            print("You can\'t add more locations to this file")
        else:
            with open(document, 'a+') as file:
                file.write('|' + location)
        file.close()

# Sorting algorithm

In [ ]:
origins = [origins for origins in output['origin_addresses']]
destinations = [destinations for destinations in output['destination_addresses']]

distance_matrix = {}

for origin in origins:
    values = []
    for element in output['rows'][origins.index(origin)]['elements']:
        kms = element['distance']['text']
        if "," in kms:
            kms = kms.replace(",", "")
        values.append(float(kms.strip('km')))
    distance_matrix.update({origin:values})
    
df = pd.DataFrame(distance_matrix, index=destinations)
df.index.name = 'destinations'

In [ ]:
df #displays the dataframe created in the cell above

# Store data in database

In [ ]:
DB = "distance_table.db"

try:
    with sql.connect(DB) as db:
        df.to_sql("distance_table", db, if_exists='append', index_label='destinations')
except sql.Error as e:
    print("Error in database", e.args[0])
except Exception as err:
    print("Other error", err)

# From SQL database to DataFrame

In [ ]:
try:
    with sql.connect(DB) as db:
        cursor = db.cursor()
        cursor.execute("SELECT * FROM distance_table")
        df2 = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])  
except sql.Error as e:
    print('Error in database', e.args[0])
except Exception as err:
    print("Other error", err)

In [ ]:
df2